In [1]:
%load_ext advent_of_code

In [2]:
import re
import math

# Day 1

In [3]:
%%aoc 1 1

answer = 0
for line in inputs.strip().split("\n"):
    num = [c for c in line if c.isdigit()]
    answer += int(num[0] + num[-1])

Execution time: 0.0017 seconds
Correct!


In [4]:
%%aoc 1 2

digits = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
digit_map = {d: i for i, d in enumerate(digits, 1)}

answer = 0
for line in inputs.strip().split("\n"):
    num = re.findall("(?=(\d|"+"|".join(digits)+"))", line)
    num = [digit_map.get(i, None) or int(i) for i in num]
    answer += num[0] * 10 + num[-1]

Execution time: 0.0053 seconds
Correct!


# Day 6

In [5]:
%%aoc 6 1
data = list(zip(*[map(int, line.split()[1:]) for line in inputs.strip().split("\n")]))
answer = 1
for t, d in data:
    answer *= t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.001 seconds
Correct!


In [6]:
%%aoc 6 2
t, d = map(int, ["".join(line.split()[1:]) for line in inputs.strip().split("\n")])
answer = t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.00081 seconds
Correct!
